In [31]:
import requests
import time
from google.cloud import pubsub

## Test collection

In [5]:
headers = {'User-Agent': 'Mozilla/5.0'}

url = 'https://www.comichron.com/monthlycomicssales/2018/2018-06.html'

response = requests.get(url, headers=headers)

response.raise_for_status()

## Manual collection

In [8]:
for i in range(2):
    !ls

comics.ipynb
comics.ipynb


In [30]:
year = 2006
month = 4

!gcloud pubsub topics publish vaqmr --message '{"collector":"web_scrape", "work_list":[{"url":"https://www.comichron.com/monthlycomicssales/{year}/1998-01.html","storage_key":"monthlycomicssales_1998-01"}]}'

messageIds:
- '882117827305425'


In [36]:
# urls = ['https://www.comichron.com/monthlycomicssales/2018/2018-06.html',
#         'https://www.comichron.com/monthlycomicssales/1998/1998-12.html']

publisher = pubsub.PublisherClient()
topic_path = publisher.topic_path('dotufp', 'vaqmr')
                                  
for year in range(1998, 2019):
    for month in range(1, 13):
        time.sleep(60)
              
        data = f'{{"collector":"web_scrape","work_list":[{{"url":"https://www.comichron.com/monthlycomicssales/{year}/{year}-{month:02d}.html","storage_key":"monthlycomicssales/{year}-{month:02d}"}}]}}'
        
        print(data)
        publisher.publish(topic_path, data=data.encode('utf-8'))


/home/dylan/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


{"collector":"web_scrape","work_list":[{"url":"https://www.comichron.com/monthlycomicssales/1998/1998-01.html","storage_key":"monthlycomicssales/1998-01"}]}
{"collector":"web_scrape","work_list":[{"url":"https://www.comichron.com/monthlycomicssales/1998/1998-02.html","storage_key":"monthlycomicssales/1998-02"}]}
{"collector":"web_scrape","work_list":[{"url":"https://www.comichron.com/monthlycomicssales/1998/1998-03.html","storage_key":"monthlycomicssales/1998-03"}]}
{"collector":"web_scrape","work_list":[{"url":"https://www.comichron.com/monthlycomicssales/1998/1998-04.html","storage_key":"monthlycomicssales/1998-04"}]}
{"collector":"web_scrape","work_list":[{"url":"https://www.comichron.com/monthlycomicssales/1998/1998-05.html","storage_key":"monthlycomicssales/1998-05"}]}
{"collector":"web_scrape","work_list":[{"url":"https://www.comichron.com/monthlycomicssales/1998/1998-06.html","storage_key":"monthlycomicssales/1998-06"}]}
{"collector":"web_scrape","work_list":[{"url":"https://www